In [5]:
import requests
import pandas as pd
import json
import time
from datetime import datetime

# --- CONSTANTES DE LA API DE SOFASCORE ---
BASE_URL_RONDAS = "https://www.sofascore.com/api/v1/unique-tournament/11653/season/57883/events/round/"
NUM_RONDAS = 30 
# NOTA: Asegúrate que esta URL coincida con tu Referer de los HEADERS si es necesario.
BASE_URL_ESTADISTICAS = "https://www.sofascore.com/event/" 

# CONJUNTO DE HEADERS COPIADOS DEL NAVEGADOR (Anti-403)
HEADERS = {
    'accept': '*/*',
    'accept-language': 'es-419,es;q=0.9',
    'cache-control': 'max-age=0',
    'if-none-match': 'W/"c8363678b6"', 
    'priority': 'u=1, i',
    'referer': 'https://www.sofascore.com/es/torneo/futbol/chile/primera-division/11653', # CRÍTICO
    'sec-ch-ua': '"Chromium";v="142", "Google Chrome";v="142", "Not_A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': 'Windows',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest', 
}
print("Configuración y HEADERS cargados.")

Configuración y HEADERS cargados.


In [6]:
# CÓDIGO A EJECUTAR EN LA CELDA 2

def obtener_datos_partidos_sofascore():
    """Descarga los resultados base y los IDs de partidos, incluyendo goles por tiempo."""
    todos_los_partidos = []
    
    # 1. Crear una sesión HTTP
    with requests.Session() as session:
        # Configurar los encabezados de la sesión (los que copiamos del navegador)
        session.headers.update(HEADERS)
        
        # 2. Visitar la página web principal para obtener cookies de sesión/validación
        print("-> Iniciando sesión para obtener cookies de validación...")
        try:
            # URL de la página de resultados (no la API)
            # Asegúrate que la URL en tu Celda 1 sea correcta.
            session.get("https://www.sofascore.com/es/torneo/futbol/chile/primera-division/11653", timeout=10) 
        except requests.exceptions.RequestException as e:
            print(f"Advertencia: Fallo al iniciar la sesión web. Error: {e}")
            return pd.DataFrame() 

        # 3. Iterar y descargar los datos de la API (usando la sesión validada)
        for ronda in range(1, NUM_RONDAS + 1):
            api_url = f"{BASE_URL_RONDAS}{ronda}"
            # LÍNEA CORREGIDA: Usamos NUM_RONDAS
            print(f"-> Descargando Ronda {ronda}/{NUM_RONDAS}...") 
            
            try:
                # Usar la sesión para la solicitud de la API
                response = session.get(api_url, timeout=15)
                response.raise_for_status()
                data = response.json()
                
                for evento in data.get('events', []):
                    if evento.get('status', {}).get('description') != 'Ended':
                        continue

                    # --- Extracción de datos (continúa como antes) ---
                    goles_p1_h = evento['homeScore'].get('period1', 0)
                    goles_p2_h = evento['homeScore'].get('period2', 0)
                    goles_p1_a = evento['awayScore'].get('period1', 0)
                    goles_p2_a = evento['awayScore'].get('period2', 0)

                    partido_info = {
                        'Ronda': ronda,
                        'ID_Partido': evento.get('id'),
                        'Fecha': datetime.fromtimestamp(evento.get('startTimestamp')).strftime('%Y-%m-%d'),
                        'Local': evento['homeTeam'].get('name'),
                        'Visitante': evento['awayTeam'].get('name'),
                        'Goles_Local_Final': evento['homeScore'].get('current'),
                        'Goles_Visitante_Final': evento['awayScore'].get('current'),
                        'Goles_Local_P1': goles_p1_h,
                        'Goles_Visitante_P1': goles_p1_a,
                        'Goles_Local_P2': goles_p2_h,
                        'Goles_Visitante_P2': goles_p2_a,
                    }
                    todos_los_partidos.append(partido_info)
                
                time.sleep(0.5) 
                
            except requests.exceptions.RequestException as e:
                print(f"Advertencia: Fallo al descargar la Ronda {ronda}. Error: {e}")
                time.sleep(5) 
                continue
                
        return pd.DataFrame(todos_los_partidos)

# Ejecución de la Fase 1
df_resultados_base = obtener_datos_partidos_sofascore()

print("\n" + "=" * 50)
print(f"✅ Extracción de la Base de Datos Exitosa. Total de partidos ENDED recolectados: {len(df_resultados_base)}")
print("=" * 50)

-> Iniciando sesión para obtener cookies de validación...
-> Descargando Ronda 1/30...
Advertencia: Fallo al descargar la Ronda 1. Error: 403 Client Error: Forbidden for url: https://www.sofascore.com/api/v1/unique-tournament/11653/season/57883/events/round/1
-> Descargando Ronda 2/30...
Advertencia: Fallo al descargar la Ronda 2. Error: 403 Client Error: Forbidden for url: https://www.sofascore.com/api/v1/unique-tournament/11653/season/57883/events/round/2
-> Descargando Ronda 3/30...
Advertencia: Fallo al descargar la Ronda 3. Error: 403 Client Error: Forbidden for url: https://www.sofascore.com/api/v1/unique-tournament/11653/season/57883/events/round/3
-> Descargando Ronda 4/30...
Advertencia: Fallo al descargar la Ronda 4. Error: 403 Client Error: Forbidden for url: https://www.sofascore.com/api/v1/unique-tournament/11653/season/57883/events/round/4
-> Descargando Ronda 5/30...
Advertencia: Fallo al descargar la Ronda 5. Error: 403 Client Error: Forbidden for url: https://www.sofas

In [7]:
# CÓDIGO A EJECUTAR EN LA CELDA 3

def obtener_estadisticas_avanzadas(row):
    """
    Descarga el JSON del partido individual y extrae las métricas clave para el ML.
    (Basado en la estructura JSON proporcionada).
    """
    event_id = row['ID_Partido']
    api_url = f"{BASE_URL_ESTADISTICAS}{event_id}/statistics" 
    
    # Valores por defecto en caso de fallo
    stats = {
        'xG_Local': None, 'xG_Visitante': None,
        'Posesion_Local': None, 'Posesion_Visitante': None,
        'Tiros_Total_Local': None, 'Tiros_Total_Visitante': None,
        'Tiros_Arco_Local': None, 'Tiros_Arco_Visitante': None,
        'Amarillas_Local': None, 'Amarillas_Visitante': None,
        'Corners_Local': None, 'Corners_Visitante': None,
    }
    
    try:
        response = requests.get(api_url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        json_data = response.json()
        
        # --- Lógica de Extracción de Estadísticas ---
        stat_groups = json_data.get('statistics', [{}])[0].get('groups', []) # Apunta al período "ALL" [0]
        
        for group in stat_groups:
            group_name = group.get('groupName')
            
            # Iterar sobre los elementos dentro de cada grupo
            for stat in group.get('statisticsItems', []):
                name = stat.get('name')
                home_val = stat.get('homeValue')
                away_val = stat.get('awayValue')
                
                # Mapeo de Features
                if name == 'Expected goals':
                    stats['xG_Local'] = home_val
                    stats['xG_Visitante'] = away_val
                elif name == 'Ball possession':
                    stats['Posesion_Local'] = home_val
                    stats['Posesion_Visitante'] = away_val
                elif name == 'Total shots':
                    stats['Tiros_Total_Local'] = home_val
                    stats['Tiros_Total_Visitante'] = away_val
                elif name == 'Shots on target':
                    stats['Tiros_Arco_Local'] = home_val
                    stats['Tiros_Arco_Visitante'] = away_val
                elif name == 'Yellow cards':
                    stats['Amarillas_Local'] = home_val
                    stats['Amarillas_Visitante'] = away_val
                elif name == 'Corner kicks':
                    stats['Corners_Local'] = home_val
                    stats['Corners_Visitante'] = awayVal
        
    except Exception as e:
        # print(f"Fallo en API de stats para ID {event_id}: {e}") # Para depuración
        pass
        
    return pd.Series(stats)

In [8]:
# CÓDIGO A EJECUTAR EN LA CELDA 4

print(f"Iniciando extracción de estadísticas avanzadas para {len(df_resultados_base)} partidos...")

# Aplicar la función a la base de datos (Esto puede tomar tiempo)
# Usamos un time.sleep para evitar la sobrecarga de la API.

def aplicar_stats(df):
    stats_list = []
    
    for index, row in df.iterrows():
        stats = obtener_estadisticas_avanzadas(row)
        stats_list.append(stats)
        
        # Pausa de cortesía
        time.sleep(0.3) 
        if (index + 1) % 50 == 0:
             print(f"Progreso: {index + 1}/{len(df)} partidos procesados.")
    
    return pd.DataFrame(stats_list)

# 1. Aplicar la función y crear un DataFrame de solo estadísticas
df_stats = aplicar_stats(df_resultados_base)

# 2. Unir la base y las estadísticas (Horizontalmente)
df_completo = pd.concat([df_resultados_base, df_stats], axis=1)

# 3. Preprocesamiento final (Creación de la variable Target)
df_completo['Fecha'] = pd.to_datetime(df_completo['Fecha'])
df_completo = df_completo.sort_values(by='Fecha').reset_index(drop=True)

def obtener_resultado(row):
    if row['Goles_Local_Final'] > row['Goles_Visitante_Final']:
        return 'Local'
    elif row['Goles_Local_Final'] < row['Goles_Visitante_Final']:
        return 'Visitante'
    return 'Empate'

df_completo['Target'] = df_completo.apply(obtener_resultado, axis=1)

# El dataset final crudo está listo
print("\n" + "=" * 50)
print("✅ Dataset Crudo Finalizado. Listo para el cálculo de Features.")
print(f"Total de Filas: {len(df_completo)}")
print("=" * 50)

# Muestra y guarda el resultado
display(df_completo[['Fecha', 'Local', 'Visitante', 'posesion_local', 'tiros_arco_local', 'Target']].tail(10))

df_completo.to_csv('dataset_sofascore_completo_crudo.csv', index=False)

Iniciando extracción de estadísticas avanzadas para 0 partidos...


KeyError: 'Fecha'

In [1]:
import pandas as pd
import requests

# --- CONSTANTES NECESARIAS (Asumiendo Celda 1 ejecutada) ---
# Si no has ejecutado la Celda 1, descomenta y pega aquí tus HEADERS y URLs.
# HEADERS = { ... tus headers anti-403 ... }
# BASE_URL_ESTADISTICAS = "https://www.sofascore.com/api/v1/event/"

# --- DATOS DE PRUEBA ---
ID_PARTIDO_PRUEBA = 11986407 # El ID de tu ejemplo de JSON
temp_data = [{'ID_Partido': ID_PARTIDO_PRUEBA, 'Ronda': 1}]
df_prueba = pd.DataFrame(temp_data)

# Ejecutar la función de la Celda 3 sobre el DataFrame de prueba
print(f"-> Probando la extracción de estadísticas para el ID: {ID_PARTIDO_PRUEBA}...")

# El método .apply() ejecuta la función y devuelve una Serie
estadisticas_obtenidas = df_prueba.apply(obtener_estadisticas_avanzadas, axis=1).iloc[0]

if estadisticas_obtenidas.isnull().all():
    print("\n" + "=" * 50)
    print("❌ Resultado de la Prueba: FALLO")
    print("   La API individual de estadísticas (Celda 3) también devolvió un error 403 o no encontró datos.")
    print("   Necesitamos cambiar de estrategia a Flashscore/Selenium.")
    print("=" * 50)
else:
    print("\n" + "=" * 50)
    print("✅ Resultado de la Prueba: ÉXITO")
    print("   La lógica de extracción de la Celda 3 es correcta. La extracción funcionó.")
    print("   Esto confirma que SOLO la API de RONDAS (Celda 2) está bloqueada.")
    print("=" * 50)
    print("\nEstadísticas del Partido:")
    print(estadisticas_obtenidas)

-> Probando la extracción de estadísticas para el ID: 11986407...


NameError: name 'obtener_estadisticas_avanzadas' is not defined

In [9]:
import requests
import pandas as pd
import json

# --- CONSTANTES Y HEADERS (CORREGIDOS) ---
ID_PARTIDO_PRUEBA = 11986407
API_URL = f"https://www.sofascore.com/api/v1/event/{ID_PARTIDO_PRUEBA}/statistics"

# HEADERS ajustados: Eliminamos 'if-none-match' y 'cache-control' para forzar la descarga completa (Status 200 OK)
HEADERS = {
    'accept': '*/*',
    'accept-language': 'es-419,es;q=0.9',
    'priority': 'u=1, i',
    'referer': 'https://www.sofascore.com/es/torneo/futbol/chile/primera-division/11653',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest', 
}

def extraer_stats_individuales(url):
    """Descarga y extrae las métricas clave de un único JSON de estadísticas."""
    stats = {
        'xG_Local': None, 'xG_Visitante': None,
        'Posesion_Local': None, 'Posesion_Visitante': None,
        'Tiros_Total_Local': None, 'Tiros_Total_Visitante': None,
        'Tiros_Arco_Local': None, 'Tiros_Arco_Visitante': None,
        'Amarillas_Local': None, 'Amarillas_Visitante': None,
        'Corners_Local': None, 'Corners_Visitante': None,
    }
    
    try:
        print(f"-> Intentando descargar: {url}")
        # Usamos los HEADERS corregidos
        response = requests.get(url, headers=HEADERS, timeout=10)
        
        print(f"   Status Code: {response.status_code}")
        response.raise_for_status() # Lanza error si es 4xx o 5xx
        
        json_data = response.json()
        
        # Acceder al período "ALL" (el primer elemento del array 'statistics')
        all_stats = json_data.get('statistics', [{}])[0]
        stat_groups = all_stats.get('groups', [])
        
        for group in stat_groups:
            for stat in group.get('statisticsItems', []):
                name = stat.get('name')
                home_val = stat.get('homeValue')
                away_val = stat.get('awayValue')
                
                # Extracción y Mapeo
                if name == 'Expected goals':
                    stats['xG_Local'] = home_val
                    stats['xG_Visitante'] = away_val
                elif name == 'Ball possession':
                    stats['Posesion_Local'] = home_val
                    stats['Posesion_Visitante'] = away_val
                elif name == 'Total shots':
                    stats['Tiros_Total_Local'] = home_val
                    stats['Tiros_Total_Visitante'] = away_val
                elif name == 'Shots on target':
                    stats['Tiros_Arco_Local'] = home_val
                    stats['Tiros_Arco_Visitante'] = away_val
                elif name == 'Yellow cards':
                    stats['Amarillas_Local'] = home_val
                    stats['Amarillas_Visitante'] = away_val
                elif name == 'Corner kicks':
                    stats['Corners_Local'] = home_val
                    stats['Corners_Visitante'] = away_val

        return stats

    except requests.exceptions.HTTPError as errh:
        print(f"❌ Error HTTP (403/404/etc): {errh}")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error de Conexión: {e}")
    except Exception as e:
        print(f"❌ Error de Extracción/JSON: {e}")
        
    return None

# --- EJECUCIÓN DEL TEST ---
datos_partido = extraer_stats_individuales(API_URL)

if datos_partido and not all(v is None for v in datos_partido.values()):
    print("\n" + "=" * 50)
    print("✅ ÉXITO: Extracción de estadísticas individuales FUNCIONANDO.")
    print("   El problema era la caché o el bloqueo de la API de Rondas.")
    print("=" * 50)
    # Crear un DataFrame para mostrar el resultado limpio
    df_resultado = pd.DataFrame([datos_partido]).T.rename(columns={0: "Valor"})
    print("\nEstadísticas Avanzadas del Partido:")
    print(df_resultado)
else:
    print("\n" + "=" * 50)
    print("❌ FALLO: La API individual TAMBIÉN está bloqueada o falló.")
    print("   Debemos cambiar a la estrategia de Flashscore/Selenium.")
    print("=" * 50)

-> Intentando descargar: https://www.sofascore.com/api/v1/event/11986407/statistics
   Status Code: 403
❌ Error HTTP (403/404/etc): 403 Client Error: Forbidden for url: https://www.sofascore.com/api/v1/event/11986407/statistics

❌ FALLO: La API individual TAMBIÉN está bloqueada o falló.
   Debemos cambiar a la estrategia de Flashscore/Selenium.
